In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from astropy.time import Time
from astropy import units as u
from simulate import ThreeBodySimulator
from coords import get_initial
from check_eclipse import check_sun_eclipse, check_moon_eclipse, detect_accurate_times
import sys
 
sys.stdout = open('record.log', mode = 'w',encoding='utf-8')

coarse_step = 1
fine_steps = 100
total_lenth = 50

def double_accuracy_simulation():
    # 初始化天体
    sun_body = get_initial("Sun")
    earth_body = get_initial("Earth")
    moon_body = get_initial("Moon")
    jupiter_body = get_initial("Jupiter")
    venus_body = get_initial("Venus")
    planet_list = [sun_body, earth_body, moon_body, jupiter_body]
    aux_planet_list = []
    # 创建模拟器（时间步长为粗步长）
    simulator = ThreeBodySimulator(
        bodies=planet_list,
        aux_list=aux_planet_list,
        dt=coarse_step
    )
    # 运行50年模拟
    print("Start coarse simulation...")
    start_time = time.time()
    trajectories, t_range = simulator.simulate_rk4(years=total_lenth)
    print("Coarse simulation time:", time.time()-start_time)
    print("Start detecting sun eclipses...")
    start_time = time.time()
    filtered_times, eclipse_types = check_sun_eclipse(trajectories['Sun'], 
                                                   trajectories['Moon'], 
                                                   trajectories['Earth'],
                                                   t_range)
    sun_eclipse_start, sun_eclipse_type, sun_eclipse_end = detect_accurate_times(check_sun_eclipse, trajectories, filtered_times, eclipse_types, fine_steps)
    print("Sun eclipse calculation time:", time.time()-start_time)
    print("Start detecting moon eclipses...")
    start_time = time.time()
    filtered_times, eclipse_types = check_moon_eclipse(trajectories['Sun'], 
                                                   trajectories['Moon'], 
                                                   trajectories['Earth'],
                                                   t_range)
    moon_eclipse_start, moon_eclipse_type, moon_eclipse_end = detect_accurate_times(check_moon_eclipse, trajectories, filtered_times, eclipse_types, fine_steps)
    print("Moon eclipse calculation time:", time.time()-start_time)
    return trajectories, t_range, sun_eclipse_start, sun_eclipse_type, sun_eclipse_end, moon_eclipse_start, moon_eclipse_type, moon_eclipse_end
        

def plot_trajectories(trajectories):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    for name, pos_list in trajectories.items():
        positions = np.array(pos_list)
        ax.plot(positions[:,0], positions[:,1], positions[:,2],
                label=name, alpha=0.7)

    ax.set_xlabel('X (AU)')
    ax.set_ylabel('Y (AU)')
    ax.set_zlabel('Z (AU)')
    ax.set_title('8 Year Orbital Trajectories')
    ax.legend()
    plt.savefig('trajectories.png')
    plt.show()

# 运行绘图
# plot_trajectories(trajectories)

START = Time(2025, format='jyear') + 6 * u.hour
print(f"Simulate solar eclipses during year 2025 and {2025+total_lenth}")
print("Initial time:", START.iso)
trajectories, t_range, sun_eclipse_start, sun_eclipse_type, sun_eclipse_end, moon_eclipse_start, moon_eclipse_type, moon_eclipse_end = double_accuracy_simulation()
print("Simulation ended. Print results:")
print("=" * 60)
print("Times of sun eclipses:")
for start_time, type, end_time in zip(sun_eclipse_start, sun_eclipse_type, sun_eclipse_end):
    print("-" * 60)
    print("Start time:", (START + start_time * coarse_step * u.hour).iso)
    print("Type:", type)
    print("Max time:", (START + (start_time + end_time) / 2 * coarse_step * u.hour).iso)
    print("End time", (START + end_time * coarse_step * u.hour).iso)
print("=" * 60)
print("Times of moon eclipses:")
for start_time, type, end_time in zip(moon_eclipse_start, moon_eclipse_type, moon_eclipse_end):
    print("-" * 60)
    print("Start time:", (START + start_time * coarse_step * u.hour).iso)
    print("Type:", type)
    print("Max time:", (START + (start_time + end_time) / 2 * coarse_step * u.hour).iso)
    print("End time", (START + end_time * coarse_step * u.hour).iso)


import json
# 保存轨道数据到文件
with open('trajectories.json', 'w') as f:
        # 将轨道数据转换为可序列化的格式
    serializable_trajectories = {}
    for body, positions in trajectories.items():
        # 将numpy数组转换为列表
        serializable_trajectories[body] = positions.tolist()
    json.dump(serializable_trajectories, f)


In [2]:
trajectories

{'Sun': array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 9.58118782e-12, -3.62696128e-12,  8.21991159e-14],
        [ 3.83249950e-11, -1.45062813e-11,  3.28779044e-13],
        ...,
        [ 5.80810730e-02,  1.16933308e-01, -2.63901679e-03],
        [ 5.80814302e-02,  1.16933409e-01, -2.63901909e-03],
        [ 5.80817875e-02,  1.16933509e-01, -2.63902139e-03]]),
 'Earth': array([[ 9.83353153e-01,  1.73472348e-17,  5.55111512e-17],
        [ 9.83352298e-01,  7.29108496e-04,  1.06206661e-08],
        [ 9.83350908e-01,  1.45821590e-03,  2.14750635e-08],
        ...,
        [ 1.04115536e+00,  9.40499021e-02, -2.60103685e-03],
        [ 1.04117151e+00,  9.47783337e-02, -2.60101669e-03],
        [ 1.04118714e+00,  9.55067765e-02, -2.60099642e-03]]),
 'Moon': array([[ 9.80875798e-01, -5.76243272e-04,  2.05079340e-04],
        [ 9.80881549e-01,  1.28871926e-04,  2.04156674e-04],
        [ 9.80886996e-01,  8.34042845e-04,  2.03214919e-04],
        ...,
        [ 1.043746

In [17]:
from jplephem.spk import SPK
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time
import astropy.units as u

def load_de440_data(start_time, time_steps, step_hours=1):
    """
    读取DE440星历表数据
    
    参数:
    start_time: astropy Time对象，起始时间
    time_steps: int, 总时间步数
    step_hours: float, 每步的小时数
    
    返回:
    dict: 包含各个天体位置的字典，格式与trajectories相同
    """
    # 加载DE421星历表
    kernel = SPK.open('de440.bsp')
    
    # 创建时间序列
    times = start_time + np.arange(time_steps) * step_hours * u.hour
    
    # 转换为Julian日期
    jd = times.jd
    
    # 初始化结果字典
    positions = {
        'Sun': [],
        'Earth': [],
        'Moon': [],
        'Jupiter': []
    }
    
    # DE440中的ID对照:
    # Sun = 10
    # Earth-Moon barycenter = 3
    # Moon (relative to Earth) = 301
    # Jupiter = 5
    
    for t in jd:
        # 获取太阳相对于太阳系重心的位置 (转换为AU)
        sun_pos = kernel[0,10].compute(t) / 149597870.7  # km to AU
        
        # 获取地球-月亮质心相对于太阳系重心的位置
        earth_moon_pos = kernel[0,3].compute(t) / 149597870.7
        
        # 获取月球相对于地球的位置
        moon_rel_earth = kernel[3,301].compute(t) / 149597870.7
        
        # 计算地球和月球的绝对位置
        earth_mass = 5.97237e24  # kg
        moon_mass = 7.342e22     # kg
        total_mass = earth_mass + moon_mass
        
        earth_pos = earth_moon_pos - (moon_mass/total_mass) * moon_rel_earth
        moon_pos = earth_pos + moon_rel_earth
        
        # 获取木星相对于太阳系重心的位置
        jupiter_pos = kernel[0,5].compute(t) / 149597870.7
        
        # 存储位置数据
        positions['Sun'].append(sun_pos)
        positions['Earth'].append(earth_pos)
        positions['Moon'].append(moon_pos)
        positions['Jupiter'].append(jupiter_pos)
    
    # 转换为numpy数组
    for body in positions:
        positions[body] = np.array(positions[body])
    
    return positions

def compare_and_plot(de440_positions, simulated_positions, start_time, step_hours=1):
    """
    比较模拟结果与DE440数据并绘图，展示位置误差随时间变化
    
    参数:
    de440_positions: dict, DE440数据
    simulated_positions: dict, 模拟数据
    start_time: astropy Time对象，起始时间
    step_hours: float, 每步的小时数
    """
    bodies = ['Sun', 'Earth', 'Moon', 'Jupiter']
    time_steps = len(simulated_positions['Sun'])
    # 将时间转换为天数
    times = np.arange(time_steps) * step_hours / 24  
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
    plt.figure(figsize=(12, 8))
    for body in bodies:
        # 计算每个时间点上的位置差的模
        diff = np.linalg.norm(
            de440_positions[body] - simulated_positions[body],
            axis=1
        )
        
        # 使用半对数坐标绘制
        plt.semilogy(times, diff, label=body)
    
    plt.xlabel('时间 (天)')
    plt.ylabel('位置误差 (AU)')
    plt.title('轨道位置误差随时间变化')
    plt.grid(True)
    plt.legend()
    plt.savefig('position_errors.png')
    plt.close()

In [14]:
# 假设您已经有了模拟得到的trajectories数据
# 使用相同的起始时间和时间步数获取DE421数据
START = Time(2025, format='jyear') + 6 * u.hour
time_steps = len(trajectories['Sun'])

# 读取DE440数据
de440_positions = load_de440_data(START, time_steps, step_hours=coarse_step)



In [15]:
de440_positions

{'Sun': array([[-0.00573061, -0.00457675, -0.00178861],
        [-0.00573031, -0.00457689, -0.00178868],
        [-0.00573001, -0.00457703, -0.00178874],
        ...,
        [ 0.00600103, -0.00010884, -0.00021863],
        [ 0.00600111, -0.00010858, -0.00021851],
        [ 0.00600119, -0.00010832, -0.0002184 ]]),
 'Earth': array([[-0.18441389,  0.88263262,  0.38280801],
        [-0.1851304 ,  0.88251016,  0.38275491],
        [-0.18584682,  0.88238722,  0.3827016 ],
        ...,
        [-0.15781216,  0.88953692,  0.38531313],
        [-0.15853023,  0.88942485,  0.38526455],
        [-0.15924821,  0.88931229,  0.38521576]]),
 'Moon': array([[-0.18340983,  0.88059995,  0.38170604],
        [-0.18410422,  0.88048696,  0.38165805],
        [-0.18479861,  0.88037367,  0.38160995],
        ...,
        [-0.15752558,  0.89202462,  0.38626161],
        [-0.15826661,  0.89191381,  0.38621562],
        [-0.15900758,  0.89180232,  0.38616935]]),
 'Jupiter': array([[ 1.05030294,  4.57425439,  1.

In [18]:

# 比较并绘图
compare_and_plot(de440_positions, trajectories, START, step_hours=coarse_step)

Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
Font 'default' does not have a glyph for '\u2212' [U+2212], substituting with a dummy symbol.
